In [1]:
import pandas as pd
import math
import time

#todo: replace with you path
compustat_dataSet = pd.read_stata('C:/Users/laura/Desktop/InnovationProject/data/geocoding_done/compustat/compustat_geocoded_2.dta')


#initialize coord map
compustat_coordMap = {}
for i in range(-180,180):
    compustat_coordMap[i] = {}
    for j in range(-90,90):#
        #we initialize a list here to collect companies in an area
        compustat_coordMap[i][j] = []

    """"
    One degree of latitude equals approximately 364,000 feet (69 miles).
    One-degree of longitude equals 288,200 feet (54.6 miles)
    So if we round, the max distance will be probably a big enough radius. We need to keep in
    mind what will happen, if some company has two different lats like -9.4999 and -9.5 
    (lets catch that later)
    
    
    look for edge cases -> how many?
    """
#fill coord map
for i in range(len(compustat_dataSet)):
    entry = compustat_dataSet.iloc[i]
    lon = round(entry['lon'])
    lat = round(entry['lat'])
    compustat_coordMap[lon][lat].append(entry)
    


#compustat_dataSet.describe()
#print(coordMapSmall[-97][33])

In [3]:
#Now do the same for the NETS dataset
NETS_dataSet = pd.read_stata('C:/Users/laura/Desktop/InnovationProject/data//geocoding_done/NETS/NETS_geocoded_cleaned_2.dta')
NETS_dataSet.shape
NETS_dataSet.head()

#initialize coord map
NETS_coordMap = {}
for i in range(-180,180):
    NETS_coordMap[i] = {}
    for j in range(-90,90):
        #we initialize a list here to collect companies in an area
        NETS_coordMap[i][j] = []

#fill coord map

for i in range(len(NETS_dataSet)):
    entry = NETS_dataSet.iloc[i]
    lon = round(entry['longNETS'])
    lat = round(entry['latNETS'])
    NETS_coordMap[lon][lat].append(entry)
    
#output dataset
output_data =  []
    

#NETS_dataSet.describe()
#smallDataSet.describe()
#print(NETS_coordMap[-71][42])
#print(len(NETS_coordMap[-71][42]))

In [4]:
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Radius of the Earth in miles (mean value)
    radius_miles = 3959.0
    
    # Calculate the distance
    distance = radius_miles * c
    
    return distance

In [5]:
def match(longitude, latitude):
    #match rounded
    for entryNETS in NETS_coordMap[longitude][latitude]:
        
        nets_lon = entryNETS['longNETS']
        nets_lat = entryNETS['latNETS']
        dist = 2 # Adjust Distance (miles) to lower results
        if(haversine_distance(comp_lat, comp_lon, nets_lat, nets_lon) <= dist): 
            output_data.append([entryComp['gvkey'], entryNETS['hqduns'], comp_lat, comp_lon, nets_lat, nets_lon])
            
    #match gvk to hqduns
    #one gvk to many hqduns
    #so there redundant gvk entries
    #also add both lat/long 
    

    

In [ ]:

final['similarity_score'] = (
    final[['hqcompany_left','hqcompany_right']]
    .apply(lambda x: difflib.SequenceMatcher(None, x[0].lower(), x[1].lower()).ratio(), axis=1)
)

output_df.to_stata('C:/Users/laura/Desktop/InnovationProject/data/geocoding_done/geomatch2.dta') 
print(output_df.info())
print(output_df.head())

In [6]:
start = time.time()
for i in range(len(compustat_dataSet)):
    entryComp = compustat_dataSet.iloc[i]
    comp_lon = entryComp['lon']
    comp_lat = entryComp['lat']
    rounded_lon = round(comp_lon)
    rounded_lat =  round(comp_lat)
    
    #match rounded
    match(rounded_lon, rounded_lat)
    
    # ~~ compute edge cases ~~   
    clo_margin = comp_lon % 1
    cla_margin = comp_lat % 1
    lon_d = (0.45 <= clo_margin < 0.5) #edge case down rounded
    lon_u = (0.5 <= clo_margin <= 0.55) #edge case up rounded
    lat_d = (0.45 <= cla_margin < 0.5) #edge case down rounded
    lat_u = (0.5 <= cla_margin <= 0.55) #edge case up rounded
    
    if(lon_d):
        match((rounded_lon + 1), rounded_lat) #u,r -> upper hash, rounded hash
        if(lat_d):
            match((rounded_lon + 1), (rounded_lat + 1)) #u,u
    elif(lon_u):
        match((rounded_lon - 1), rounded_lat) #l,r -> lower hash, rounded hash
        if(lat_u):
            match((rounded_lon - 1), (rounded_lat - 1)) #l,l
        
    if(lat_d):
        match(rounded_lon, (rounded_lat + 1)) #r,u
        if(lon_u):
            match((rounded_lon - 1), (rounded_lat + 1)) #l,u
    elif(lat_u):
        match(rounded_lon, (rounded_lat - 1)) #r,l
        if(lon_d):
            match((rounded_lon + 1), (rounded_lat - 1)) #u,l
    # ~~ compute edge cases ~~

print(" runtime: ", time.time() - start)

# ---- OUTPUT DATA ------
output_df = pd.DataFrame(output_data, columns=['gvkey', 'hqduns', 'latComp', 'lonComp', 'latNETS', 'lonNETS'])
#todo: replace with you path
output_df.to_stata('C:/Users/laura/Desktop/InnovationProject/data/geocoding_done/geomatch2.dta') 
print(output_df.info())
print(output_df.head())

 runtime:  298.29191064834595
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2790383 entries, 0 to 2790382
Data columns (total 6 columns):
 #   Column   Dtype  
---  ------   -----  
 0   gvkey    int32  
 1   hqduns   int32  
 2   latComp  float64
 3   lonComp  float64
 4   latNETS  float32
 5   lonNETS  float32
dtypes: float32(2), float64(2), int32(2)
memory usage: 85.2 MB
None
   gvkey   hqduns    latComp    lonComp    latNETS    lonNETS
0   1050  2412869  33.007766 -96.828911  33.030899 -96.832298
1   1050  2578310  33.007766 -96.828911  33.015800 -96.817001
2   1050  2778129  33.007766 -96.828911  32.986099 -96.835899
3   1050  3506710  33.007766 -96.828911  33.001701 -96.841904
4   1050  3523417  33.007766 -96.828911  32.980801 -96.838303


In [ ]:
final = pd.read_stata('C:/Users/laura/Desktop/InnovationProject/data/geocoding_done/geomatch_names.dta',index_col='gvkey')


final['similarity_score'] = (
    final[['hqcompany_left','hqcompany_right']]
    .apply(lambda x: difflib.SequenceMatcher(None, x[0].lower(), x[1].lower()).ratio(), axis=1)
)
